# Azure AI SearchとSemantic Kernelを使用したAgentic RAG

このノートブックでは、Azure AI SearchをベクトルデータベースとしてSemantic Kernelと統合し、Retrieval-Augmented Generation（RAG）を実装する方法を学びます。

**このノートブックで達成できること：**
*   Azure AI Searchから情報を取得してユーザーの質問に答えるAIエージェントを構築します。
*   複数のツール（プラグイン）を組み合わせて、より複雑な問い合わせに対応する方法を理解します。
*   Semantic Kernelを使用した基本的なAgentic RAGパターンの実装を体験します。

**実行順序：**
このノートブックのセルは、上から順番に実行してください。各セルは前のセルの結果に依存しています。

## 環境の初期化

### パッケージのインストール

まず、このノートブックで必要なPythonパッケージをインストールします。これには、Azure AI Search、OpenAI、Semantic Kernelを操作するためのライブラリが含まれます。

In [1]:
import importlib
import subprocess
import sys

# チェックするライブラリ名と、インストールするパッケージ名のマッピング
packages = {
    "azure.search.documents": "azure-search-documents==11.4.0",
    "dotenv": "python-dotenv",
    "openai": "openai",
    "semantic_kernel": "semantic-kernel",
    "azure.identity": "azure-identity"
}

not_installed = []
for lib, pkg in packages.items():
    try:
        # ライブラリがインポート可能か試す
        importlib.import_module(lib.split('.')[0])
    except ImportError:
        # インポートできなければインストールリストに追加
        not_installed.append(pkg)

if not_installed:
    print(f"不足しているパッケージをインストールします: {', '.join(not_installed)}")
    # pipコマンドを実行してインストール
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet"] + not_installed)
else:
    print("必要なパッケージはすべてインストール済みです。")

必要なパッケージはすべてインストール済みです。


### パッケージのインポート
以下のコードは必要なパッケージをインポートします：

In [2]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### Semantic KernelとAIサービスの作成

ここでは、AIエージェントの「頭脳」となる部分を設定します。具体的には、Semantic Kernelが大規模言語モデル（LLM）と通信するための準備を行います。

次のセルでは、以下の2つの重要な処理を実行します。

1. **.envファイルからAPIキーを読み込む:** `load_dotenv()` を使って、ノートブックに直接書き込むことなく、安全にAPIキーなどの秘密情報を環境変数として読み込みます。
2. **チャット補完サービスの作成:** `OpenAIChatCompletion` を使って、LLMと対話するための「サービス」を作成します。これは、Semantic Kernelがプロンプトを送信し、モデルからの応答を受け取るための窓口となります。このサービスをエージェントに組み込むことで、エージェントは思考し、応答を生成する能力を得ます。

In [3]:
load_dotenv()
# 非同期OpenAIクライアントを初期化
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# OpenAI ChatCompletion サービスを作成
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### プラグインの定義

ここでは、AIエージェントが使用する「ツール」を定義します。プラグインは、エージェントに特定の機能を追加するコンポーネントです。

次の2つのセルでは、以下のプラグインを作成します：

1. **SearchPlugin（検索プラグイン）:** Azure AI Searchからドキュメントを検索し、取得した情報をもとに拡張プロンプトを構築する機能を提供します。これにより、エージェントは外部の知識ベースから関連情報を取得できるようになります。

2. **WeatherInfoPlugin（気温情報プラグイン）:** 旅行先の平均気温を提供する簡単なプラグインです。これは、エージェントが複数のツールを組み合わせて使用する方法を示すためのサンプルです。

これらのプラグインを組み合わせることで、エージェントは単純な質問応答を超えて、複数の情報源から情報を収集し、統合された回答を提供できるようになります。これがAgentic RAGの核となる機能です。

In [4]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="取得したコンテキストや関数の結果を用いて、拡張プロンプトを構築します。",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"取得したコンテキスト:\n{retrieval_context}\n\n"
            f"ユーザーのクエリ: {query}\n\n"
            "まず取得したコンテキストを確認し、それでクエリに答えられない場合は、利用可能なプラグイン関数を呼び出して答えを探してください。利用可能なコンテキストがない場合は、そのように伝えてください。"
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Azure Searchサービスからドキュメントを取得します。",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"ドキュメント: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "結果が見つかりませんでした"

In [5]:
class WeatherInfoPlugin:
    """旅行先の平均気温を提供するプラグインです。"""

    def __init__(self):
        # 旅行先とその平均気温の辞書
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="特定の旅行先の平均気温を取得します。")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "指定された旅行先の平均気温を返します。"]:
        """旅行先の平均気温を取得します。"""
        # 入力された旅行先を正規化（小文字に変換）
        normalized_destination = destination.lower()

        # 旅行先の気温を検索
        if normalized_destination in self.destination_temperatures:
            return f"{destination}の平均気温は{self.destination_temperatures[normalized_destination]}です。"
        else:
            return f"申し訳ありませんが、{destination}の気温情報はありません。利用可能な旅行先は、モルディブ、スイスアルプス、アフリカのサファリです。"

## ベクトルデータベースの初期化

ここでは、AIエージェントが情報を検索するための「知識ベース」を準備します。このステップは、Agentic RAGシステムの核となる重要な部分です。

**このセクションで行うこと：**

1. **Azure AI Searchサービスの接続:** 
   - `.env`ファイルから認証情報を読み込み、Azure AI Searchサービスに接続します
   - このサービスは、文書を保存し、後で検索するためのクラウドベースのデータベースです

2. **インデックスの作成（データベーステーブルの準備）:**
   - `travel-documents`という名前のインデックスを作成します
   - 各文書には`id`（識別番号）と`content`（内容）フィールドが含まれます

3. **サンプル文書の追加:**
   - Contoso Travelに関する5つのサンプル文書をデータベースに追加します
   - これらの文書は、後でエージェントが質問に答える際の情報源となります

**RAGにおける重要性：**
このベクトルデータベースにより、エージェントは単純な事前学習された知識だけでなく、最新の企業情報や特定のドメイン知識にもアクセスできるようになります。これが「Retrieval-Augmented Generation」の「Retrieval（検索）」部分です。

In [6]:
# 永続ストレージを使用してAzure AI Searchを初期化
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# インデックススキーマを定義
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# インデックスが既に存在するかチェック、存在しない場合は作成
try:
    existing_index = index_client.get_index(index_name)
    print(f"インデックス '{index_name}' は既に存在します。既存のインデックスを使用します。")
except Exception:
    # インデックスが存在しない場合は作成
    print(f"新しいインデックス '{index_name}' を作成中...")
    index_client.create_index(index)


# 拡張されたサンプル文書
documents = [
    {"id": "1", "content": "Contoso Travelは世界中のエキゾチックな目的地への豪華な休暇パッケージを提供します。"},
    {"id": "2", "content": "私たちのプレミアム旅行サービスには、パーソナライズされた旅程計画と24時間365日のコンシェルジュサポートが含まれます。"},
    {"id": "3", "content": "Contosoの旅行保険は医療緊急事態、旅行キャンセル、手荷物紛失をカバーします。"},
    {"id": "4", "content": "人気の旅行先には、モルディブ、スイスアルプス、アフリカのサファリなどがあります。"},
    {"id": "5", "content": "Contoso Travelはブティックホテルへの独占アクセスとプライベートガイドツアーを提供します。"}
]

# インデックスに文書を追加
search_client.upload_documents(documents)

インデックス 'travel-documents' は既に存在します。既存のインデックスを使用します。


In [7]:
# AIエージェントを作成
# これまでに準備したコンポーネントを組み合わせて、実際のAIエージェントを構築します
agent = ChatCompletionAgent(
    service=chat_completion_service,  # LLMサービス（GPT-4o-mini）
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],  # 使用するプラグイン
    name="TravelAgent",  # エージェントの名前
    instructions="提供されたツールとコンテキストを使用して旅行に関する質問に答えてください。コンテキストが提供されている場合は、「そのコンテキストはありません」と言わないでください。",  # エージェントの動作指示
)

### AIエージェントの実行とストリーミング表示

ここでは、作成したAIエージェントを実際に動作させ、リアルタイムで応答を表示する方法を学びます。

**このセクションで行うこと：**

1. **会話スレッドの作成:** エージェントとの対話を管理するためのスレッドを作成します
2. **サンプル質問の実行:** 3つの異なるタイプの質問を順番に実行します
3. **ストリーミング表示:** エージェントの応答をリアルタイムで表示し、関数呼び出しの過程も確認できます

**実行される質問：**
- Contosoの旅行保険について（検索機能のテスト）
- モルディブの平均気温について（気温プラグインのテスト）
- Contosoが提供する寒い旅行先とその気温について（複数プラグインの組み合わせテスト）

このデモにより、Agentic RAGシステムがどのように複数の情報源から情報を収集し、統合された回答を提供するかを確認できます。

In [8]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    # テスト用のユーザー入力
    user_inputs = [
        "Contosoの旅行保険について説明してください",
        "モルディブの平均気温を教えてください",
        "Contosoが提供する寒い旅行先で良いところはありますか？その平均気温も教えてください",
    ]

    for user_input in user_inputs:
        # ユーザー入力をHTML形式で表示
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>ユーザー:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # ストリーミング関数呼び出しを再構築するためのバッファ
        current_function_name = None
        argument_buffer = ""

        # エージェントをストリーミングモードで実行
        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # 引数を蓄積（チャンクでストリーミングされる）
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # 結果を表示する前に保留中の関数呼び出しを完了
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # 生の文字列として残す

                        function_calls.append(f"関数呼び出し: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\n関数結果:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # 関数呼び出しがあった場合は折りたたみ可能な形式で表示
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>関数呼び出し (クリックして展開)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        # エージェントの応答を表示
        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'アシスタント'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

# メイン関数を実行
await main()

以下のような出力が表示されるはずです：

**質問1: Contosoの旅行保険について説明してください**

```text
ユーザー: 
Contosoの旅行保険について説明してください

TravelAgent:

Contosoの旅行保険は、以下のようなカバーを提供しています：

1. **医療緊急事態** - 旅行中に病気や怪我が発生した際の医療費をカバーします。
2. **旅行キャンセル** - 旅行をキャンセルする必要が生じた場合の費用を補償します。
3. **手荷物紛失** - 荷物が紛失した場合の損失をカバーします。

この保険により、旅行者は安心して旅を楽しむことができます。
```

**質問2: モルディブの平均気温を教えてください**

```text
ユーザー: 
モルディブの平均気温を教えてください

TravelAgent:

モルディブの平均気温は約28°C（82°F）です。一年を通して温暖な熱帯気候で、ビーチリゾートとして人気の旅行先です。
```

**質問3: Contosoが提供する寒い旅行先で良いところはありますか？その平均気温も教えてください**

```text
ユーザー: 
Contosoが提供する寒い旅行先で良いところはありますか？その平均気温も教えてください

TravelAgent:

Contosoが提供する寒い旅行先として、スイスアルプスがおすすめです。スイスアルプスの平均気温は約7°C（45°F）で、美しい山岳風景とウィンタースポーツを楽しむことができます。
```

**注意事項:**
関数呼び出しの表示は、エージェントが実際にプラグイン関数を呼び出した場合にのみ表示されます。シンプルな質問の場合、エージェントは事前学習された知識で回答する場合があります。より複雑な質問や、明確に検索や特定情報の取得が必要な質問をした場合に、関数呼び出しが表示される可能性があります。

## まとめ

このノートブックでは、Azure AI SearchとSemantic Kernelを使用したAgentic RAGシステムの基本実装を学びました。

**学習内容：**
- **ベクトルデータベース（Azure AI Search）** でドキュメントを検索・取得
- **複数のプラグイン** （SearchPlugin、WeatherInfoPlugin）を組み合わせた情報統合
- **ストリーミング応答** でリアルタイムな対話体験を実現

**重要なポイント：**
エージェントは単一の情報源に依存せず、複数のツールを自律的に使い分けて包括的な回答を生成します。これが従来のRAGを超えた「Agentic RAG」の特徴です。

**次のステップ：**
より複雑なビジネスシナリオや、エンタープライズレベルのセキュリティ・スケーラビリティを考慮した実装に挑戦してみましょう。